In [1]:
import string
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
from sklearn import metrics

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')